In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
def load_data(file_name):
    """Returns a pandas dataframe from a execl file."""
    return pd.read_excel(file_name)
df = load_data('/Superstore Dataset.xlsx')

In [34]:
time_series_data= df[['Order Date', 'Category','Product Name','Quantity' ]]

In [35]:
time_series_data.head()

,Order Date,Category,Product Name,Quantity
0,2019-01-03,Office Supplies,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",2
1,2019-01-04,Office Supplies,GBC Standard Plastic Binding Systems Combs,2
2,2019-01-04,Office Supplies,Avery 508,3
3,2019-01-04,Office Supplies,SAFCO Boltless Steel Shelving,3
4,2019-01-05,Office Supplies,Avery Hi-Liter EverBold Pen Style Fluorescent ...,3


In [37]:
time_series_data = time_series_data.rename(columns={'Order Date': 'ds', 'Quantity': 'y'})

In [10]:
'''spec = [['Category'],
        ['Category', 'Sub-Category'],
        ['Category', 'Sub-Category', 'Product Name']]
        '''

In [43]:
time_series_data = time_series_data.sort_values(by=['Category', 'Product Name','ds'])

# Identify the last two values for each product sorted by date to create test data
last_two_values = time_series_data.groupby(['Category', 'Product Name']).tail(2)

# Remove the last two values from the original train data
time_series_data = time_series_data.drop(last_two_values.index)

# Create a validation dataset for the removed values
val_data = last_two_values

In [59]:
spec = [['Category'],
        ['Category', 'Product Name']]

In [ ]:
!pip install hierarchicalforecast


In [54]:
time_series_data

,ds,Category,Product Name,y
12983,2014-12-02,Furniture,12-1/2 Diameter Round Wall Clock,5
12913,2014-12-06,Furniture,12-1/2 Diameter Round Wall Clock,3
19025,2015-11-15,Furniture,12-1/2 Diameter Round Wall Clock,2
10948,2015-12-07,Furniture,12-1/2 Diameter Round Wall Clock,4
12735,2016-04-30,Furniture,12-1/2 Diameter Round Wall Clock,7
...,...,...,...,...
21399,2021-09-23,Technology,netTALK DUO VoIP Telephone Service,2
21991,2021-12-05,Technology,netTALK DUO VoIP Telephone Service,4
22917,2022-06-20,Technology,netTALK DUO VoIP Telephone Service,3
8049,2022-07-03,Technology,netTALK DUO VoIP Telephone Service,4


In [60]:
spec

[['Category'], ['Category', 'Product Name']]

In [61]:
from hierarchicalforecast.utils import aggregate

train_agg, S_train, tags = aggregate(time_series_data, spec)
valid_agg, _, _ = aggregate(val_data, spec)

In [46]:
train_agg

,ds,y
unique_id,,
Furniture,2014-01-06,9
Furniture,2014-01-07,3
Furniture,2014-01-10,1
Furniture,2014-01-11,2
Furniture,2014-01-13,9
...,...,...
Technology/netTALK DUO VoIP Telephone Service,2021-09-23,2
Technology/netTALK DUO VoIP Telephone Service,2021-12-05,4
Technology/netTALK DUO VoIP Telephone Service,2022-06-20,3


In [ ]:
!pip install statsforecast

In [47]:
from statsforecast import StatsForecast
from statsforecast.models import HoltWinters
from statsforecast.models import SimpleExponentialSmoothing


# Simple Exponential smoothing is done as there is no seasonal data
model = StatsForecast(
    models=[SimpleExponentialSmoothing(alpha=0.5)],
    freq='D',
    n_jobs=-1
)

model.fit(train_agg)

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:619: FutureWarning: Passing unique_id as the index is deprecated. Please provide it as a column instead.
  warnings.warn(


StatsForecast(models=[SES])

In [48]:
#predict the next 2 forecast for the models
p = model.forecast(h=2, fitted=True)
p_fitted = model.forecast_fitted_values()

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:467: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:485: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:485: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [62]:
S_train.shape

(1856, 1853)

In [ ]:
#reconi
from hierarchicalforecast.methods import BottomUp, TopDown, MinTrace, ERM, OptimalCombination
from hierarchicalforecast.core import HierarchicalReconciliation

reconcilers = [BottomUp(),
               TopDown(method='forecast_proportions'),
               TopDown(method='average_proportions'),
               TopDown(method='proportion_averages'),
               MinTrace(method='ols', nonnegative=True),
               MinTrace(method='wls_struct', nonnegative=True),
               MinTrace(method='wls_var', nonnegative=True),
               MinTrace(method='mint_shrink', nonnegative=True),
               MinTrace(method='mint_cov', nonnegative=True),
               OptimalCombination(method='ols', nonnegative=True),
               OptimalCombination(method='wls_struct', nonnegative=True),
               ERM(method='closed'),
               ERM(method='reg'),
               ERM(method='reg_bu'),
              ]

rec_model = HierarchicalReconciliation(reconcilers=reconcilers)

p_rec = rec_model.reconcile(Y_hat_df=p, Y_df=p_fitted, S=regularized_cov_matrix, tags=tags)

In [ ]:
p_rec_ = p_rec.merge(valid_agg, on=['ds', 'unique_id'], how='left')
p_rec_['y'] = p_rec_['y'].fillna(0)
p_rec_ = p_rec_.reset_index(drop=False)

from sklearn.metrics import mean_squared_error

rmse = dict()
for model_ in p_rec.columns[1:]:
    rmse_ = mean_squared_error(p_rec_['y'].values, p_rec_[model_].values, squared=False)/1e3
    # get only the model name
    model__ = model_.split('/')[-1]
    rmse[model__] = rmse_
pd.DataFrame(rmse, index=['RMSE']).T.sort_values('RMSE')